#### Code implementation using functions

In [1]:
import multiprocessing
import cv2
import time
import logging

In [2]:
BUFFER_SIZE = 150
VIDEO_PATH = '/home/acer/workspace/CV-Local/utils/multiple-video-analytics-POC/test-videos/video-1.mp4'
FACE_CASCADE_PATH = './models/haarcascade_frontalface_default.xml'

In [3]:
def setup_logging():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_face_cascade_classifier():
    face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)
    if face_cascade.empty():
        logging.error('Failed to load the face cascade classifier.')
    return face_cascade

def detect_faces(frames, window_name, result_list, manager, face_cascade):
    while True:
        shared_result_list = manager.list(result_list[0])

        if len(frames) > 0:
            frame = frames[-1]
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

            object_prefix = 'face_'
            object_dict = {}
            for (x, y, w, h) in faces:
                object_dict[object_prefix] = [(x, y, w, h)]
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

            shared_result_list.append((window_name, object_dict))
            result_list[0] = shared_result_list

            if len(result_list[0]) > BUFFER_SIZE:
                shared_result_list.pop(0)

            cv2.imshow(window_name, frame)

            key = cv2.waitKey(1)
            if key == 27:
                cv2.destroyAllWindows()
                break
        else:
            logging.warning("No frames present")
            cv2.destroyAllWindows()

def record_frames(frames):
    cap = cv2.VideoCapture(VIDEO_PATH)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frames.append(frame.copy())
        if len(frames) > BUFFER_SIZE:
            frames.pop(0)

        cv2.imshow("inside_record_frames", frame)

        key = cv2.waitKey(1)
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [4]:
if __name__ == '__main__':
    
    setup_logging()
    
    with multiprocessing.Manager() as manager:
        frames = manager.list([])
        result_list = manager.list([[], []])

        face_cascade = load_face_cascade_classifier()

        process_record_frames = multiprocessing.Process(target=record_frames, args=(frames,))
        process_record_frames.daemon = True
        process_record_frames.start()

        process_detect_faces = multiprocessing.Process(target=detect_faces, args=(frames, "face_detection_window", result_list, manager, face_cascade))
        process_detect_faces.start()

        process_record_frames.join()
        process_detect_faces.join()


2023-11-24 12:11:07,854 - WARNING - No frames present
2023-11-24 12:11:07,859 - WARNING - No frames present
2023-11-24 12:11:07,861 - WARNING - No frames present
2023-11-24 12:11:07,864 - WARNING - No frames present
2023-11-24 12:11:07,866 - WARNING - No frames present
2023-11-24 12:11:07,868 - WARNING - No frames present
2023-11-24 12:11:07,869 - WARNING - No frames present
2023-11-24 12:11:07,871 - WARNING - No frames present
2023-11-24 12:11:07,873 - WARNING - No frames present
2023-11-24 12:11:07,876 - WARNING - No frames present
2023-11-24 12:11:07,878 - WARNING - No frames present
2023-11-24 12:11:07,879 - WARNING - No frames present
2023-11-24 12:11:07,881 - WARNING - No frames present
2023-11-24 12:11:07,884 - WARNING - No frames present
2023-11-24 12:11:07,885 - WARNING - No frames present


2023-11-24 12:11:07,887 - WARNING - No frames present
2023-11-24 12:11:07,889 - WARNING - No frames present
2023-11-24 12:11:07,891 - WARNING - No frames present
2023-11-24 12:11:07,893 - WARNING - No frames present
2023-11-24 12:11:07,895 - WARNING - No frames present
2023-11-24 12:11:07,897 - WARNING - No frames present
2023-11-24 12:11:07,899 - WARNING - No frames present
2023-11-24 12:11:07,904 - WARNING - No frames present
2023-11-24 12:11:07,906 - WARNING - No frames present
2023-11-24 12:11:07,908 - WARNING - No frames present
2023-11-24 12:11:07,911 - WARNING - No frames present
2023-11-24 12:11:07,914 - WARNING - No frames present
2023-11-24 12:11:07,917 - WARNING - No frames present
2023-11-24 12:11:07,919 - WARNING - No frames present
2023-11-24 12:11:07,922 - WARNING - No frames present
2023-11-24 12:11:07,925 - WARNING - No frames present


#### Code improvements - added manager context with lock

In [5]:
import multiprocessing
import cv2
import time
import logging


In [6]:
VIDEO_PATH = '/home/acer/workspace/CV-Local/utils/multiple-video-analytics-POC/test-videos/video-1.mp4'
FACE_CASCADE_PATH = './models/haarcascade_frontalface_default.xml'
BUFFER_SIZE = 150

In [7]:
def setup_logging():
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_face_cascade_classifier():
    face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)
    if face_cascade.empty():
        logging.error('Failed to load the face cascade classifier.')
    return face_cascade

def detect_faces(frames, window_name, result_list, manager, face_cascade):
    while True:
        shared_result_list = manager.list(result_list[0])

        with manager.Lock():
            if frames:
                frame = frames[-1]
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

                object_prefix = 'face_'
                object_dict = {}
                for (x, y, w, h) in faces:
                    object_dict[object_prefix] = [(x, y, w, h)]
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

                shared_result_list.append((window_name, object_dict))
                result_list[0] = shared_result_list

                if len(result_list[0]) > BUFFER_SIZE:
                    shared_result_list.pop(0)

                cv2.imshow(window_name, frame)

                key = cv2.waitKey(1)
                if key == 27:
                    cv2.destroyAllWindows()
                    break
            else:
                logging.warning("No frames present")
                cv2.destroyAllWindows()

def record_frames(frames):
    cap = cv2.VideoCapture(VIDEO_PATH)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        with manager.Lock():
            frames.append(frame.copy())
            if len(frames) > BUFFER_SIZE:
                frames.pop(0)

            cv2.imshow("inside_record_frames", frame)

            key = cv2.waitKey(1)
            if key == 27:
                break

    cap.release()
    cv2.destroyAllWindows()

In [8]:
if __name__ == '__main__':
    setup_logging()

    with multiprocessing.Manager() as manager:
        frames = manager.list([])
        result_list = manager.list([[], []])

        face_cascade = load_face_cascade_classifier()

        process_record_frames = multiprocessing.Process(target=record_frames, args=(frames,))
        process_record_frames.daemon = True
        process_record_frames.start()

        process_detect_faces = multiprocessing.Process(target=detect_faces, args=(frames, "face_detection_window", result_list, manager, face_cascade))
        process_detect_faces.start()

        process_record_frames.join()
        process_detect_faces.join()


2023-11-24 12:11:10,159 - WARNING - No frames present
2023-11-24 12:11:10,166 - WARNING - No frames present
2023-11-24 12:11:10,170 - WARNING - No frames present
2023-11-24 12:11:10,173 - WARNING - No frames present
2023-11-24 12:11:10,177 - WARNING - No frames present
2023-11-24 12:11:10,181 - WARNING - No frames present
2023-11-24 12:11:10,183 - WARNING - No frames present
2023-11-24 12:11:10,186 - WARNING - No frames present
2023-11-24 12:11:10,190 - WARNING - No frames present
2023-11-24 12:11:10,193 - WARNING - No frames present
2023-11-24 12:11:10,196 - WARNING - No frames present
2023-11-24 12:11:10,199 - WARNING - No frames present
2023-11-24 12:11:10,201 - WARNING - No frames present
2023-11-24 12:11:10,204 - WARNING - No frames present
2023-11-24 12:11:10,207 - WARNING - No frames present
2023-11-24 12:11:10,210 - WARNING - No frames present
2023-11-24 12:11:10,212 - WARNING - No frames present
2023-11-24 12:11:10,216 - WARNING - No frames present
2023-11-24 12:11:10,219 - WA

2023-11-24 12:11:10,222 - WARNING - No frames present
2023-11-24 12:11:10,225 - WARNING - No frames present
2023-11-24 12:11:10,228 - WARNING - No frames present


#### Code improvements - class encapsulation

In [10]:
import multiprocessing
import cv2
import logging

In [11]:
VIDEO_PATH = '/home/acer/workspace/CV-Local/utils/multiple-video-analytics-POC/test-videos/video-1.mp4'
FACE_CASCADE_PATH = '/home/acer/workspace/CV-Local/utils/multiple-video-analytics-POC/models/haarcascade_frontalface_default.xml'
BUFFER_SIZE = 150

In [12]:
class FaceDetectionProcess(multiprocessing.Process):
    def __init__(self, frames, window_name, result_list, manager, face_cascade, exit_flag, frames_available_event):
        super(FaceDetectionProcess, self).__init__()
        self.frames = frames
        self.window_name = window_name
        self.result_list = result_list
        self.manager = manager
        self.face_cascade = face_cascade
        self.exit_flag = exit_flag
        self.frames_available_event = frames_available_event

    def run(self):
        while not self.exit_flag.is_set():
            self.frames_available_event.wait()

            with self.manager.Lock():
                if self.frames:
                    frame = self.frames[-1]
                    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    faces = self.face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

                    object_prefix = 'face_'
                    object_dict = {}
                    for (x, y, w, h) in faces:
                        object_dict[object_prefix] = [(x, y, w, h)]
                        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 0), 2)

                    shared_result_list = self.manager.list(self.result_list[0])
                    shared_result_list.append((self.window_name, object_dict))
                    self.result_list[0] = shared_result_list

                    if len(self.result_list[0]) > BUFFER_SIZE:
                        shared_result_list.pop(0)

                    logging.info("Faces detected and added to result list.")

                    cv2.imshow(self.window_name, frame)

                    key = cv2.waitKey(1)
                    if key == 27:
                        cv2.destroyAllWindows()
                        self.exit_flag.set()
                        break
                else:
                    logging.warning("No frames present")
                    cv2.destroyAllWindows()
                    self.exit_flag.set()
                    break

            self.frames_available_event.clear()

        logging.info("FaceDetectionProcess terminated.")

class RecordFramesProcess(multiprocessing.Process):
    def __init__(self, frames, manager, exit_flag, frames_available_event):
        super(RecordFramesProcess, self).__init__()
        self.frames = frames
        self.manager = manager
        self.exit_flag = exit_flag
        self.frames_available_event = frames_available_event

    def run(self):
        cap = cv2.VideoCapture(VIDEO_PATH)

        while not self.exit_flag.is_set():
            ret, frame = cap.read()
            if not ret:
                logging.error("Error reading frame from video stream.")
                break

            with self.manager.Lock():
                self.frames.append(frame.copy())
                logging.info("Frame read and added to frames list.")

                if len(self.frames) > BUFFER_SIZE:
                    self.frames.pop(0)

            self.frames_available_event.set()

            cv2.imshow("inside_record_frames", frame)

            key = cv2.waitKey(1)
            if key == 27:
                self.exit_flag.set()
                break

        cap.release()
        cv2.destroyAllWindows()

        logging.info("RecordFramesProcess terminated.")

In [13]:
if __name__ == '__main__':
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    with multiprocessing.Manager() as manager:
        frames = manager.list([])
        result_list = manager.list([[], []])
        exit_flag = manager.Event()
        frames_available_event = manager.Event()

        face_cascade = cv2.CascadeClassifier(FACE_CASCADE_PATH)
        if face_cascade.empty():
            logging.error('Failed to load the face cascade classifier.')

        process_record_frames = RecordFramesProcess(frames, manager, exit_flag, frames_available_event)
        process_record_frames.daemon = True
        process_record_frames.start()

        process_detect_faces = FaceDetectionProcess(
            frames, "face_detection_window", result_list, manager, face_cascade, exit_flag, frames_available_event
        )
        process_detect_faces.start()

        process_record_frames.join()
        process_detect_faces.join()

    logging.info("Application terminated.")


2023-11-24 12:11:13,050 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,158 - INFO - Faces detected and added to result list.
2023-11-24 12:11:13,177 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,208 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,232 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,257 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,286 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,298 - INFO - Faces detected and added to result list.
2023-11-24 12:11:13,313 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,343 - INFO - Faces detected and added to result list.
2023-11-24 12:11:13,362 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,392 - INFO - Frame read and added to frames list.
2023-11-24 12:11:13,402 - INFO - Faces detected and added to result list.
2023-11-24 12:11:13,421 - INFO - Frame read and added to frames list.
2023